In [203]:
import numpy as np

In [204]:
# |0> = [1,0], |1> = [0,1], kronecker product used to form composite state basis
H = np.array([[1,1],[1,-1]])/np.sqrt(2)

In [205]:
def tensorProduct(lst):
    """
    Given a list of numpy arrays (or lists) finds the total tensor product 
    """
    prod = np.array(lst[0],dtype=np.complex_)
    for l in lst[1:]:
        prod = np.kron(prod,np.array(l,dtype=np.complex_))
    return prod

def getState(i,n):
    """
    Gets the state corresponding to i in the binary convention (with n bits)
    """
    i = format(i,"b")
    
    if n-len(i) >= 0:
        s = tensorProduct([np.array([1,0]) for j in range(n-len(i))])
    else:
        i = i[:n]
        s = 1

    for d in i:
        if d == "0":
            s = np.kron(s,np.array([1,0]))
        elif d == "1":
            s = np.kron(s,np.array([0,1]))
    return s

def getNumber(s): 
    """
    Function that finds the associated number for a qubit tensor product state. 
    """
    N = s.shape[0]
    n = int(np.log2(N))
    for i in range(n):
        if np.allclose(getState(i,n) @ s, 1):
            return i

def extendOperator(A,i,n):
    """
    Takes a qubit operator and extends it to a n qubit space, the operator created acts on the ith tensor product.
    """
    return tensorProduct([np.eye(2) if j != i else A for j in range(n)])

In [206]:
def invQFT(f):
    """
    Performs inverse quantum fourier transform on qubit states.
    """

    N = f.shape[0]
    n = int(np.log2(N))
    fInv = f

    for i in range(n): 
        for k in range(2,n-i):
            cRk = np.eye(2,dtype=np.complex_)
            cRk[1,1] = np.exp(-2*np.pi*complex(0,1)/(2**k))
            cRkE = extendOperator(cRk,i,n)
            fInv = cRkE @ fInv
        
        #apply H
        HE = extendOperator(H,i,n)
        fInv = HE @ fInv
    
    return fInv


In [207]:
#test case for invQFT: 3 qubit system, |2> = |010> state should be the result
f = tensorProduct([[1,np.exp(np.pi*complex(0,0.5))],[1,-1],[1,1]])/np.sqrt(8) 
print(getNumber(invQFT(f)))


2


In [208]:

def phaseEstimation(t,U,e): 
    """
    Performs phase estimation.

    Inputs:
    t - accuracy of psi returned
    U - unitary matrix
    e - normalised ket
    """
    N = U.shape[0]

    #cache?
    def pow(n,prev=np.eye(N)):
        next = U @ prev
        if n == 1:
            return next
        return pow(n-1,next)
    

    f = tensorProduct([[1,0] for i in range(t)])


    for i in range(t):
        HE = extendOperator(H,i,t)
        f = HE @ f

        #apply 
        Ui = np.eye(2)
        Ui[1,1] = e @ pow(2**i) @ e
        UiE = extendOperator(Ui,i,t)
        f = UiE @ f
        
    return getNumber(invQFT(f)) #apply inverse QFT


In [209]:
#test for phase esimation, identity matrix, result should be 1
U1 = np.array([[1,0],[0,1]])
e1 = np.array([1,0])
phaseEstimation(2,U1,e1)

0

In [210]:



def findOrder(a,N,n,m,t): 
    """
    a - some int (will be removed as a parameter eventually)
    N - number of qubits used 
    n - number to be factorized 
    m - number of common factors found
    """
    #use basis |0> = [1,0,0,..], |1> = [0,1,0,..], ...
    #define U_a
    U_a = np.zeros([2**N,2**N])
    for i in range(2**N):
        for j in range(2**N):
            if i == a*j % n:
                U_a[i,j] = 1
    
    #in the actual algorithm |1> is plugged in, however due to model QM problablities the eigenstate is randomly chosen
    eigenVal, eigenVec = np.linalg.eig(U_a)
    rs = np.empty(m) 
    for i in range(m):
        index = np.random.randint(0,high=2**N)
        rs[i] = index/phaseEstimation(t,U_a,eigenVec[index])
        
    #find LCM
    R = np.lcm.reduce(rs)
    
    return R

    